In [1]:
from interface import ObjectInterface
import numpy as np
import utils

FILENAME_DATABASE = 'data/database.db'

def get_interface() -> ObjectInterface:
    interface = ObjectInterface(FILENAME_DATABASE)
    interface.connect()
    return interface

In [2]:
with get_interface() as interface:
    obj = interface.get_object(1)
    print(obj.attributes)
    birthday = obj['birthday']
    print(birthday, type(birthday))

{'first_name': 'Anne', 'last_name': 'Fischer', 'birthday': '1997-03-25'}
1997-03-25 <class 'datetime.date'>


In [2]:
with get_interface() as interface:
    obj = interface.create_object(interface.get_class(name='ArrayData'), some_numbers=np.arange(10).reshape((2, -1)))
    interface.commit()
    array = obj['some_numbers']
    print(array)

Error executing access transformer: Failed to interpret file <_io.BytesIO object at 0x0000020C5697BA10> as a pickle
None


In [4]:
arr = np.arange(10).reshape((2, -1))
arr_bytes = utils.array_to_bytes(arr)
print(arr)
print(arr_bytes)

loaded_arr = utils.bytes_to_array(arr_bytes)
print(loaded_arr)

[[0 1 2 3 4]
 [5 6 7 8 9]]
b"\x93NUMPY\x01\x00v\x00{'descr': '<i4', 'fortran_order': False, 'shape': (2, 5), }                                                          \n\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00\x00\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00\x00\x08\x00\x00\x00\t\x00\x00\x00"
[[0 1 2 3 4]
 [5 6 7 8 9]]
